In [1]:
import numpy as np 
from PIL import Image
import matplotlib.pyplot as plt

import torch 
from torch import nn
from torchvision import models

from utils import ImageTransform,make_datapath_list,HymenopteraDataset

train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

size = 224
mean = (0.485,0.456,0.406)
std = (0.229,0.224,0.225)

train_dataset = HymenopteraDataset(file_list = train_list,transform = ImageTransform(size,mean,std),phase="train")

val_dataset = HymenopteraDataset(file_list = val_list,transform=ImageTransform(size,mean,std),phase = "val")

batch_size = 8

train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size=batch_size,shuffle=False)

dataloaders_dict = {"train":train_dataloader,"val":val_dataloader}

./hymenoptera_data/train/**/*.jpg
./hymenoptera_data/val/**/*.jpg


In [2]:
print(torch.cuda.is_available())

True


In [3]:
use_pretrained = True

net = models.vgg16(pretrained = use_pretrained)

net.classifier[6] = nn.Linear(in_features=4096,out_features=2)

net.train()


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
criterion = nn.CrossEntropyLoss()

In [5]:
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

update_params_name_1 = ["features"]
update_params_name_2 = ["classifier.0.weight","classifier.0.bias","classifier.3.weight","classifier.3.bias"]
update_params_name_3 = ["classifier.6.weight","classifier.6.bias"]

for name,param in net.named_parameters():
    if update_params_name_1[0] in name:
        param_reqauires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1に格納:",name)

    elif name in update_params_name_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2に格納",name)
    
    elif name in update_params_name_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3に格納",name)

    else:
        param.requires_grad = False
        print("購買計算なし",name)



params_to_update_1に格納: features.0.weight
params_to_update_1に格納: features.0.bias
params_to_update_1に格納: features.2.weight
params_to_update_1に格納: features.2.bias
params_to_update_1に格納: features.5.weight
params_to_update_1に格納: features.5.bias
params_to_update_1に格納: features.7.weight
params_to_update_1に格納: features.7.bias
params_to_update_1に格納: features.10.weight
params_to_update_1に格納: features.10.bias
params_to_update_1に格納: features.12.weight
params_to_update_1に格納: features.12.bias
params_to_update_1に格納: features.14.weight
params_to_update_1に格納: features.14.bias
params_to_update_1に格納: features.17.weight
params_to_update_1に格納: features.17.bias
params_to_update_1に格納: features.19.weight
params_to_update_1に格納: features.19.bias
params_to_update_1に格納: features.21.weight
params_to_update_1に格納: features.21.bias
params_to_update_1に格納: features.24.weight
params_to_update_1に格納: features.24.bias
params_to_update_1に格納: features.26.weight
params_to_update_1に格納: features.26.bias
params_to_update_1に格納: f

In [6]:
from torch import optim

optimizer = optim.SGD([
    {"params":params_to_update_1,"lr":1e-4},
    {"params":params_to_update_2,"lr":5e-4},
    {"params":params_to_update_3,"lr":1e-3}
],momentum=0.9)

In [7]:
from tqdm import tqdm

def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
    print("使用デバイス:",device)

    net.to(device)

    #torch.backends.cudnn.benchmark = True

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch + 1,num_epochs))
        print("-------------------------")

        for phase in ["train","val"]:
            if phase == "train":
                net.train()
            else:
                net.eval()

            epoch_loss = 0.0
            epoch_corrects = 0

            if (epoch == 0) and (phase=="train"):
                continue

            for inputs,labels in tqdm(dataloaders_dict[phase]):

                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = net(inputs)
                    loss = criterion(outputs,labels)
                    _,preds = torch.max(outputs,1)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)

                    

                epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
                epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

                print("{} Loss:{:.4f} Acc: {:.4f}".format(phase,epoch_loss,epoch_acc))

In [8]:
num_epochs = 30

train_model(net,dataloaders_dict,criterion,optimizer,num_epochs)

使用デバイス: cuda:0
  0%|          | 0/10 [00:00&lt;?, ?it/s]Epoch 1/30
-------------------------
  0%|          | 0/16 [00:00&lt;?, ?it/s]val Loss:0.0507 Acc: 0.4444
Epoch 2/30
-------------------------
  6%|▋         | 1/16 [00:01&lt;00:16,  1.13s/it]


RuntimeError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 3.94 GiB total capacity; 2.51 GiB already allocated; 36.31 MiB free; 2.56 GiB reserved in total by PyTorch)

In [9]:
save_path = "./weights/weight1_5.pth"
torch.save(net.state_dict(),save_path)